In [17]:
import pandas as pd 
import matplotlib.pyplot as plt
from scipy import stats as sps
import dcor

In [2]:
df = pd.read_csv('nba_game_data_updated.csv')
grouped_games = df.groupby('game_id')
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
pd.set_option('display.max_columns', None)
df = df.loc[:, ~df.columns.str.contains('_opp')]
df.head(4)

,FG_basic_totals,FGA_basic_totals,FG%_basic_totals,3P_basic_totals,3PA_basic_totals,3P%_basic_totals,FT_basic_totals,FTA_basic_totals,FT%_basic_totals,ORB_basic_totals,DRB_basic_totals,TRB_basic_totals,AST_basic_totals,STL_basic_totals,BLK_basic_totals,TOV_basic_totals,PF_basic_totals,PTS_basic_totals,MP_advanced_totals,TS%_advanced_totals,eFG%_advanced_totals,3PAr_advanced_totals,FTr_advanced_totals,ORB%_advanced_totals,DRB%_advanced_totals,TRB%_advanced_totals,AST%_advanced_totals,STL%_advanced_totals,BLK%_advanced_totals,TOV%_advanced_totals,USG%_advanced_totals,ORtg_advanced_totals,DRtg_advanced_totals,FG_basic_max,FGA_basic_max,FG%_basic_max,3P_basic_max,3PA_basic_max,3P%_basic_max,FT_basic_max,FTA_basic_max,FT%_basic_max,ORB_basic_max,DRB_basic_max,TRB_basic_max,AST_basic_max,STL_basic_max,BLK_basic_max,TOV_basic_max,PF_basic_max,PTS_basic_max,GmSc_basic_max,+/-_basic_max,TS%_advanced_max,eFG%_advanced_max,3PAr_advanced_max,FTr_advanced_max,ORB%_advanced_max,DRB%_advanced_max,TRB%_advanced_max,AST%_advanced_max,STL%_advanced_max,BLK%_advanced_max,TOV%_advanced_max,USG%_advanced_max,ORtg_advanced_max,DRtg_advanced_max,Team,Q1,Q2,Q3,Q4,Total,home,season,date,OT5,OT6,OT7,OT8,game_id,win
0,32.0,72.0,0.444,9.0,19.0,0.474,22.0,26.0,0.846,6.0,32.0,38.0,20.0,9.0,8.0,16.0,27.0,95.0,240.0,0.569,0.507,0.264,0.361,18.8,84.2,54.3,62.5,10.0,15.1,16.1,100.0,106.0,99.3,6.0,16.0,0.600,3.0,6.0,1.000,9.0,10.0,1.000,3.0,10.0,11.0,10.0,3.0,3.0,3.0,5.0,23.0,18.0,11.0,0.725,0.700,1.000,4.0,13.6,33.3,20.7,52.7,4.7,8.2,28.7,25.2,125.0,107.0,BOS,21,30,21,23,95,0,2010,2009-10-27,0.0,0.0,0.0,0.0,2009-10-27_BOS_CLE,1
1,29.0,70.0,0.414,6.0,17.0,0.353,25.0,32.0,0.781,6.0,26.0,32.0,17.0,7.0,9.0,14.0,22.0,89.0,240.0,0.529,0.457,0.243,0.457,15.8,81.3,45.7,58.6,7.9,17.0,14.3,100.0,99.9,106.6,12.0,22.0,0.545,4.0,9.0,0.667,10.0,13.0,1.000,5.0,10.0,10.0,8.0,2.0,4.0,5.0,5.0,38.0,31.2,3.0,0.685,0.636,0.409,1.0,19.2,52.1,23.8,52.9,8.7,8.1,25.0,36.0,121.0,115.0,CLE,28,17,20,24,89,1,2010,2009-10-27,0.0,0.0,0.0,0.0,2009-10-27_BOS_CLE,0
2,39.0,84.0,0.464,4.0,13.0,0.308,20.0,24.0,0.833,9.0,37.0,46.0,19.0,6.0,4.0,10.0,29.0,102.0,240.0,0.539,0.488,0.155,0.286,22.5,77.1,52.3,48.7,6.7,6.9,9.6,100.0,113.3,101.1,10.0,21.0,1.000,2.0,4.0,0.667,8.0,9.0,1.000,3.0,9.0,10.0,9.0,2.0,2.0,3.0,5.0,29.0,20.6,16.0,1.025,1.000,0.750,1.0,10.3,36.0,19.6,42.6,4.8,4.7,45.0,33.8,158.0,112.0,WAS,21,35,21,25,102,0,2010,2009-10-27,0.0,0.0,0.0,0.0,2009-10-27_DAL_WAS,1
3,30.0,76.0,0.395,4.0,18.0,0.222,27.0,34.0,0.794,11.0,31.0,42.0,16.0,6.0,9.0,10.0,20.0,91.0,240.0,0.500,0.421,0.237,0.447,22.9,77.5,47.7,53.3,6.7,12.7,9.9,100.0,101.1,113.3,10.0,25.0,1.000,2.0,6.0,0.500,12.0,13.0,0.923,4.0,8.0,9.0,6.0,2.0,3.0,3.0,4.0,34.0,21.5,7.0,0.862,1.000,0.500,3.0,12.8,30.7,14.0,37.5,6.9,8.7,25.0,41.6,213.0,123.0,DAL,21,29,24,17,91,1,2010,2009-10-27,0.0,0.0,0.0,0.0,2009-10-27_DAL_WAS,0


In [3]:
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values(by=["date"])
df.head(4)
home_df = df[df['home'] == 1].copy()
home_game_ids = home_df['game_id']
home_df = home_df.add_prefix('home_')
home_df['game_id'] = home_game_ids.values
away_df = df[df['home'] == 0].copy()
away_game_ids = away_df['game_id']
away_df = away_df.add_prefix('away_')
away_df['game_id'] = away_game_ids.values
df = pd.merge(home_df, away_df, on='game_id')
df = df.drop(columns=['home_home', 'away_home', 'home_game_id', 'away_season', 'away_date', 'away_game_id', 'away_win'])
df = df.rename(columns={'home_season': 'season', 'home_date': 'date'})
df.head(4)

,home_FG_basic_totals,home_FGA_basic_totals,home_FG%_basic_totals,home_3P_basic_totals,home_3PA_basic_totals,home_3P%_basic_totals,home_FT_basic_totals,home_FTA_basic_totals,home_FT%_basic_totals,home_ORB_basic_totals,home_DRB_basic_totals,home_TRB_basic_totals,home_AST_basic_totals,home_STL_basic_totals,home_BLK_basic_totals,home_TOV_basic_totals,home_PF_basic_totals,home_PTS_basic_totals,home_MP_advanced_totals,home_TS%_advanced_totals,home_eFG%_advanced_totals,home_3PAr_advanced_totals,home_FTr_advanced_totals,home_ORB%_advanced_totals,home_DRB%_advanced_totals,home_TRB%_advanced_totals,home_AST%_advanced_totals,home_STL%_advanced_totals,home_BLK%_advanced_totals,home_TOV%_advanced_totals,home_USG%_advanced_totals,home_ORtg_advanced_totals,home_DRtg_advanced_totals,home_FG_basic_max,home_FGA_basic_max,home_FG%_basic_max,home_3P_basic_max,home_3PA_basic_max,home_3P%_basic_max,home_FT_basic_max,home_FTA_basic_max,home_FT%_basic_max,home_ORB_basic_max,home_DRB_basic_max,home_TRB_basic_max,home_AST_basic_max,home_STL_basic_max,home_BLK_basic_max,home_TOV_basic_max,home_PF_basic_max,home_PTS_basic_max,home_GmSc_basic_max,home_+/-_basic_max,home_TS%_advanced_max,home_eFG%_advanced_max,home_3PAr_advanced_max,home_FTr_advanced_max,home_ORB%_advanced_max,home_DRB%_advanced_max,home_TRB%_advanced_max,home_AST%_advanced_max,home_STL%_advanced_max,home_BLK%_advanced_max,home_TOV%_advanced_max,home_USG%_advanced_max,home_ORtg_advanced_max,home_DRtg_advanced_max,home_Team,home_Q1,home_Q2,home_Q3,home_Q4,home_Total,season,date,home_OT5,home_OT6,home_OT7,home_OT8,home_win,game_id,away_FG_basic_totals,away_FGA_basic_totals,away_FG%_basic_totals,away_3P_basic_totals,away_3PA_basic_totals,away_3P%_basic_totals,away_FT_basic_totals,away_FTA_basic_totals,away_FT%_basic_totals,away_ORB_basic_totals,away_DRB_basic_totals,away_TRB_basic_totals,away_AST_basic_totals,away_STL_basic_totals,away_BLK_basic_totals,away_TOV_basic_totals,away_PF_basic_totals,away_PTS_basic_totals,away_MP_advanced_totals,away_TS%_advanced_totals,away_eFG%_advanced_totals,away_3PAr_advanced_totals,away_FTr_advanced_totals,away_ORB%_advanced_totals,away_DRB%_advanced_totals,away_TRB%_advanced_totals,away_AST%_advanced_totals,away_STL%_advanced_totals,away_BLK%_advanced_totals,away_TOV%_advanced_totals,away_USG%_advanced_totals,away_ORtg_advanced_totals,away_DRtg_advanced_totals,away_FG_basic_max,away_FGA_basic_max,away_FG%_basic_max,away_3P_basic_max,away_3PA_basic_max,away_3P%_basic_max,away_FT_basic_max,away_FTA_basic_max,away_FT%_basic_max,away_ORB_basic_max,away_DRB_basic_max,away_TRB_basic_max,away_AST_basic_max,away_STL_basic_max,away_BLK_basic_max,away_TOV_basic_max,away_PF_basic_max,away_PTS_basic_max,away_GmSc_basic_max,away_+/-_basic_max,away_TS%_advanced_max,away_eFG%_advanced_max,away_3PAr_advanced_max,away_FTr_advanced_max,away_ORB%_advanced_max,away_DRB%_advanced_max,away_TRB%_advanced_max,away_AST%_advanced_max,away_STL%_advanced_max,away_BLK%_advanced_max,away_TOV%_advanced_max,away_USG%_advanced_max,away_ORtg_advanced_max,away_DRtg_advanced_max,away_Team,away_Q1,away_Q2,away_Q3,away_Q4,away_Total,away_OT5,away_OT6,away_OT7,away_OT8
0,29.0,70.0,0.414,6.0,17.0,0.353,25.0,32.0,0.781,6.0,26.0,32.0,17.0,7.0,9.0,14.0,22.0,89.0,240.0,0.529,0.457,0.243,0.457,15.8,81.3,45.7,58.6,7.9,17.0,14.3,100.0,99.9,106.6,12.0,22.0,0.545,4.0,9.0,0.667,10.0,13.0,1.000,5.0,10.0,10.0,8.0,2.0,4.0,5.0,5.0,38.0,31.2,3.0,0.685,0.636,0.409,1.000,19.2,52.1,23.8,52.9,8.7,8.1,25.0,36.0,121.0,115.0,CLE,28,17,20,24,89,2010,2009-10-27,0.0,0.0,0.0,0.0,0,2009-10-27_BOS_CLE,32.0,72.0,0.444,9.0,19.0,0.474,22.0,26.0,0.846,6.0,32.0,38.0,20.0,9.0,8.0,16.0,27.0,95.0,240.0,0.569,0.507,0.264,0.361,18.8,84.2,54.3,62.5,10.0,15.1,16.1,100.0,106.0,99.3,6.0,16.0,0.6,3.0,6.0,1.000,9.0,10.0,1.0,3.0,10.0,11.0,10.0,3.0,3.0,3.0,5.0,23.0,18.0,11.0,0.725,0.700,1.00,4.000,13.6,33.3,20.7,52.7,4.7,8.2,28.7,25.2,125.0,107.0,BOS,21,30,21,23,95,0.0,0.0,0.0,0.0
1,30.0,76.0,0.395,4.0,18.0,0.222,27.0,34.0,0.794,11.0,31.0,42

In [4]:
df["Team_A"] = df.apply(lambda row: min(row["home_Team"], row["away_Team"]), axis=1)
df["Team_B"] = df.apply(lambda row: max(row["home_Team"], row["away_Team"]), axis=1)
df["matchup_id"] = df["Team_A"] + "_vs_" + df["Team_B"]
df.head(1)

,home_FG_basic_totals,home_FGA_basic_totals,home_FG%_basic_totals,home_3P_basic_totals,home_3PA_basic_totals,home_3P%_basic_totals,home_FT_basic_totals,home_FTA_basic_totals,home_FT%_basic_totals,home_ORB_basic_totals,home_DRB_basic_totals,home_TRB_basic_totals,home_AST_basic_totals,home_STL_basic_totals,home_BLK_basic_totals,home_TOV_basic_totals,home_PF_basic_totals,home_PTS_basic_totals,home_MP_advanced_totals,home_TS%_advanced_totals,home_eFG%_advanced_totals,home_3PAr_advanced_totals,home_FTr_advanced_totals,home_ORB%_advanced_totals,home_DRB%_advanced_totals,home_TRB%_advanced_totals,home_AST%_advanced_totals,home_STL%_advanced_totals,home_BLK%_advanced_totals,home_TOV%_advanced_totals,home_USG%_advanced_totals,home_ORtg_advanced_totals,home_DRtg_advanced_totals,home_FG_basic_max,home_FGA_basic_max,home_FG%_basic_max,home_3P_basic_max,home_3PA_basic_max,home_3P%_basic_max,home_FT_basic_max,home_FTA_basic_max,home_FT%_basic_max,home_ORB_basic_max,home_DRB_basic_max,home_TRB_basic_max,home_AST_basic_max,home_STL_basic_max,home_BLK_basic_max,home_TOV_basic_max,home_PF_basic_max,home_PTS_basic_max,home_GmSc_basic_max,home_+/-_basic_max,home_TS%_advanced_max,home_eFG%_advanced_max,home_3PAr_advanced_max,home_FTr_advanced_max,home_ORB%_advanced_max,home_DRB%_advanced_max,home_TRB%_advanced_max,home_AST%_advanced_max,home_STL%_advanced_max,home_BLK%_advanced_max,home_TOV%_advanced_max,home_USG%_advanced_max,home_ORtg_advanced_max,home_DRtg_advanced_max,home_Team,home_Q1,home_Q2,home_Q3,home_Q4,home_Total,season,date,home_OT5,home_OT6,home_OT7,home_OT8,home_win,game_id,away_FG_basic_totals,away_FGA_basic_totals,away_FG%_basic_totals,away_3P_basic_totals,away_3PA_basic_totals,away_3P%_basic_totals,away_FT_basic_totals,away_FTA_basic_totals,away_FT%_basic_totals,away_ORB_basic_totals,away_DRB_basic_totals,away_TRB_basic_totals,away_AST_basic_totals,away_STL_basic_totals,away_BLK_basic_totals,away_TOV_basic_totals,away_PF_basic_totals,away_PTS_basic_totals,away_MP_advanced_totals,away_TS%_advanced_totals,away_eFG%_advanced_totals,away_3PAr_advanced_totals,away_FTr_advanced_totals,away_ORB%_advanced_totals,away_DRB%_advanced_totals,away_TRB%_advanced_totals,away_AST%_advanced_totals,away_STL%_advanced_totals,away_BLK%_advanced_totals,away_TOV%_advanced_totals,away_USG%_advanced_totals,away_ORtg_advanced_totals,away_DRtg_advanced_totals,away_FG_basic_max,away_FGA_basic_max,away_FG%_basic_max,away_3P_basic_max,away_3PA_basic_max,away_3P%_basic_max,away_FT_basic_max,away_FTA_basic_max,away_FT%_basic_max,away_ORB_basic_max,away_DRB_basic_max,away_TRB_basic_max,away_AST_basic_max,away_STL_basic_max,away_BLK_basic_max,away_TOV_basic_max,away_PF_basic_max,away_PTS_basic_max,away_GmSc_basic_max,away_+/-_basic_max,away_TS%_advanced_max,away_eFG%_advanced_max,away_3PAr_advanced_max,away_FTr_advanced_max,away_ORB%_advanced_max,away_DRB%_advanced_max,away_TRB%_advanced_max,away_AST%_advanced_max,away_STL%_advanced_max,away_BLK%_advanced_max,away_TOV%_advanced_max,away_USG%_advanced_max,away_ORtg_advanced_max,away_DRtg_advanced_max,away_Team,away_Q1,away_Q2,away_Q3,away_Q4,away_Total,away_OT5,away_OT6,away_OT7,away_OT8,Team_A,Team_B,matchup_id
0,29.0,70.0,0.414,6.0,17.0,0.353,25.0,32.0,0.781,6.0,26.0,32.0,17.0,7.0,9.0,14.0,22.0,89.0,240.0,0.529,0.457,0.243,0.457,15.8,81.3,45.7,58.6,7.9,17.0,14.3,100.0,99.9,106.6,12.0,22.0,0.545,4.0,9.0,0.667,10.0,13.0,1.0,5.0,10.0,10.0,8.0,2.0,4.0,5.0,5.0,38.0,31.2,3.0,0.685,0.636,0.409,1.0,19.2,52.1,23.8,52.9,8.7,8.1,25.0,36.0,121.0,115.0,CLE,28,17,20,24,89,2010,2009-10-27,0.0,0.0,0.0,0.0,0,2009-10-27_BOS_CLE,32.0,72.0,0.444,9.0,19.0,0.474,22.0,26.0,0.846,6.0,32.0,38.0,20.0,9.0,8.0,16.0,27.0,95.0,240.0,0.569,0.507,0.264,0.361,18.8,84.2,54.3,62.5,10.0,15.1,16.1,100.0,106.0,99.3,6.0,16.0,0.6,3.0,6.0,1.0,9.0,10.0,1.0,3.0,10.0,11.0,10.0,3.0,3.0,3.0,5.0,23.0,18.0,11.0,0.725,0.7,1.0,4.0,13.6,33.3,20.7,52.7,4.7,8.2,28.7,25.2,125.0,107.0,BOS,21,30,21,23,95,0.0,0.0,0.0,0.0,BOS,CLE,BOS_vs_CLE


In [5]:
def assign_team_stats(row):
    if row['home_Team'] == row['Team_A']:
        team_a_stats = row.filter(like='home_').rename(lambda x: x.replace('home_', 'Team_A_'))
        team_b_stats = row.filter(like='away_').rename(lambda x: x.replace('away_', 'Team_B_'))
    else:
        team_a_stats = row.filter(like='away_').rename(lambda x: x.replace('away_', 'Team_A_'))
        team_b_stats = row.filter(like='home_').rename(lambda x: x.replace('home_', 'Team_B_'))
    return pd.concat([team_a_stats, team_b_stats])
team_stats_df = df.apply(assign_team_stats, axis=1)
df = pd.concat([df[['Team_A', 'Team_B', 'matchup_id', 'date', 'season', 'home_Team']], team_stats_df], axis=1)


In [6]:
df = df.drop(columns=['Team_A_win', 'Team_B_win'])
df['Team_A_win'] = df.apply(lambda row: 1 if row['Team_A_PTS_basic_totals'] > row['Team_B_PTS_basic_totals'] else 0, axis=1)
df.head(5)

/var/folders/k0/szzfrvxj5h349vrf8y8j_wg00000gn/T/ipykernel_11325/2013128443.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['Team_A_win'] = df.apply(lambda row: 1 if row['Team_A_PTS_basic_totals'] > row['Team_B_PTS_basic_totals'] else 0, axis=1)


,Team_A,Team_B,matchup_id,date,season,home_Team,Team_A_+/-_basic_max,Team_A_3P%_basic_max,Team_A_3P%_basic_totals,Team_A_3PA_basic_max,Team_A_3PA_basic_totals,Team_A_3PAr_advanced_max,Team_A_3PAr_advanced_totals,Team_A_3P_basic_max,Team_A_3P_basic_totals,Team_A_AST%_advanced_max,Team_A_AST%_advanced_totals,Team_A_AST_basic_max,Team_A_AST_basic_totals,Team_A_BLK%_advanced_max,Team_A_BLK%_advanced_totals,Team_A_BLK_basic_max,Team_A_BLK_basic_totals,Team_A_DRB%_advanced_max,Team_A_DRB%_advanced_totals,Team_A_DRB_basic_max,Team_A_DRB_basic_totals,Team_A_DRtg_advanced_max,Team_A_DRtg_advanced_totals,Team_A_FG%_basic_max,Team_A_FG%_basic_totals,Team_A_FGA_basic_max,Team_A_FGA_basic_totals,Team_A_FG_basic_max,Team_A_FG_basic_totals,Team_A_FT%_basic_max,Team_A_FT%_basic_totals,Team_A_FTA_basic_max,Team_A_FTA_basic_totals,Team_A_FT_basic_max,Team_A_FT_basic_totals,Team_A_FTr_advanced_max,Team_A_FTr_advanced_totals,Team_A_GmSc_basic_max,Team_A_MP_advanced_totals,Team_A_ORB%_advanced_max,Team_A_ORB%_advanced_totals,Team_A_ORB_basic_max,Team_A_ORB_basic_totals,Team_A_ORtg_advanced_max,Team_A_ORtg_advanced_totals,Team_A_OT5,Team_A_OT6,Team_A_OT7,Team_A_OT8,Team_A_PF_basic_max,Team_A_PF_basic_totals,Team_A_PTS_basic_max,Team_A_PTS_basic_totals,Team_A_Q1,Team_A_Q2,Team_A_Q3,Team_A_Q4,Team_A_STL%_advanced_max,Team_A_STL%_advanced_totals,Team_A_STL_basic_max,Team_A_STL_basic_totals,Team_A_TOV%_advanced_max,Team_A_TOV%_advanced_totals,Team_A_TOV_basic_max,Team_A_TOV_basic_totals,Team_A_TRB%_advanced_max,Team_A_TRB%_advanced_totals,Team_A_TRB_basic_max,Team_A_TRB_basic_totals,Team_A_TS%_advanced_max,Team_A_TS%_advanced_totals,Team_A_Team,Team_A_Total,Team_A_USG%_advanced_max,Team_A_USG%_advanced_totals,Team_A_eFG%_advanced_max,Team_A_eFG%_advanced_totals,Team_B_+/-_basic_max,Team_B_3P%_basic_max,Team_B_3P%_basic_totals,Team_B_3PA_basic_max,Team_B_3PA_basic_totals,Team_B_3PAr_advanced_max,Team_B_3PAr_advanced_totals,Team_B_3P_basic_max,Team_B_3P_basic_totals,Team_B_AST%_advanced_max,Team_B_AST%_advanced_totals,Team_B_AST_basic_max,Team_B_AST_basic_totals,Team_B_BLK%_advanced_max,Team_B_BLK%_advanced_totals,Team_B_BLK_basic_max,Team_B_BLK_basic_totals,Team_B_DRB%_advanced_max,Team_B_DRB%_advanced_totals,Team_B_DRB_basic_max,Team_B_DRB_basic_totals,Team_B_DRtg_advanced_max,Team_B_DRtg_advanced_totals,Team_B_FG%_basic_max,Team_B_FG%_basic_totals,Team_B_FGA_basic_max,Team_B_FGA_basic_totals,Team_B_FG_basic_max,Team_B_FG_basic_totals,Team_B_FT%_basic_max,Team_B_FT%_basic_totals,Team_B_FTA_basic_max,Team_B_FTA_basic_totals,Team_B_FT_basic_max,Team_B_FT_basic_totals,Team_B_FTr_advanced_max,Team_B_FTr_advanced_totals,Team_B_GmSc_basic_max,Team_B_MP_advanced_totals,Team_B_ORB%_advanced_max,Team_B_ORB%_advanced_totals,Team_B_ORB_basic_max,Team_B_ORB_basic_totals,Team_B_ORtg_advanced_max,Team_B_ORtg_advanced_totals,Team_B_OT5,Team_B_OT6,Team_B_OT7,Team_B_OT8,Team_B_PF_basic_max,Team_B_PF_basic_totals,Team_B_PTS_basic_max,Team_B_PTS_basic_totals,Team_B_Q1,Team_B_Q2,Team_B_Q3,Team_B_Q4,Team_B_STL%_advanced_max,Team_B_STL%_advanced_totals,Team_B_STL_basic_max,Team_B_STL_basic_totals,Team_B_TOV%_advanced_max,Team_B_TOV%_advanced_totals,Team_B_TOV_basic_max,Team_B_TOV_basic_totals,Team_B_TRB%_advanced_max,Team_B_TRB%_advanced_totals,Team_B_TRB_basic_max,Team_B_TRB_basic_totals,Team_B_TS%_advanced_max,Team_B_TS%_advanced_totals,Team_B_Team,Team_B_Total,Team_B_USG%_advanced_max,Team_B_USG%_advanced_totals,Team_B_eFG%_advanced_max,Team_B_eFG%_advanced_totals,Team_A_win
0,BOS,CLE,BOS_vs_CLE,2009-10-27,2010,CLE,11.0,1.0,0.474,6.0,19.0,1.0,0.264,3.0,9.0,52.7,62.5,10.0,20.0,8.2,15.1,3.0,8.0,33.3,84.2,10.0,32.0,107.0,99.3,0.6,0.444,16.0,72.0,6.0,32.0,1.000,0.846,10.0,26.0,9.0,22.0,4.000,0.361,18.0,240.0,13.6,18.8,3.0,6.0,125.0,106.0,0.0,0.0,0.0,0.0,5.0,27.0,23.0,95.0,21,30,21,23,4.7,10.0,3.0,9.0,28.7,16.1,3.0,16.0,20.7,54.3,11.0,38.0,0.725,0.569,BOS,95,25.2,100.0,0.700,0.507,3.0,0.667,0.353,9.0,17.0,0.409,0.243,4.0,6.0,52.9,58.6,8.0,17.0,8.1,17.0,4.0,9.0,52.1,81.3,10.0,

In [7]:
df = df.copy()
df['Team_A_home'] = df.apply(lambda row: 1 if row['home_Team'] == row['Team_A'] else 0, axis=1)
df=df.drop(columns=['home_Team', 'Team_A_Team', 'Team_B_Team'])
df.head(2)

,Team_A,Team_B,matchup_id,date,season,Team_A_+/-_basic_max,Team_A_3P%_basic_max,Team_A_3P%_basic_totals,Team_A_3PA_basic_max,Team_A_3PA_basic_totals,Team_A_3PAr_advanced_max,Team_A_3PAr_advanced_totals,Team_A_3P_basic_max,Team_A_3P_basic_totals,Team_A_AST%_advanced_max,Team_A_AST%_advanced_totals,Team_A_AST_basic_max,Team_A_AST_basic_totals,Team_A_BLK%_advanced_max,Team_A_BLK%_advanced_totals,Team_A_BLK_basic_max,Team_A_BLK_basic_totals,Team_A_DRB%_advanced_max,Team_A_DRB%_advanced_totals,Team_A_DRB_basic_max,Team_A_DRB_basic_totals,Team_A_DRtg_advanced_max,Team_A_DRtg_advanced_totals,Team_A_FG%_basic_max,Team_A_FG%_basic_totals,Team_A_FGA_basic_max,Team_A_FGA_basic_totals,Team_A_FG_basic_max,Team_A_FG_basic_totals,Team_A_FT%_basic_max,Team_A_FT%_basic_totals,Team_A_FTA_basic_max,Team_A_FTA_basic_totals,Team_A_FT_basic_max,Team_A_FT_basic_totals,Team_A_FTr_advanced_max,Team_A_FTr_advanced_totals,Team_A_GmSc_basic_max,Team_A_MP_advanced_totals,Team_A_ORB%_advanced_max,Team_A_ORB%_advanced_totals,Team_A_ORB_basic_max,Team_A_ORB_basic_totals,Team_A_ORtg_advanced_max,Team_A_ORtg_advanced_totals,Team_A_OT5,Team_A_OT6,Team_A_OT7,Team_A_OT8,Team_A_PF_basic_max,Team_A_PF_basic_totals,Team_A_PTS_basic_max,Team_A_PTS_basic_totals,Team_A_Q1,Team_A_Q2,Team_A_Q3,Team_A_Q4,Team_A_STL%_advanced_max,Team_A_STL%_advanced_totals,Team_A_STL_basic_max,Team_A_STL_basic_totals,Team_A_TOV%_advanced_max,Team_A_TOV%_advanced_totals,Team_A_TOV_basic_max,Team_A_TOV_basic_totals,Team_A_TRB%_advanced_max,Team_A_TRB%_advanced_totals,Team_A_TRB_basic_max,Team_A_TRB_basic_totals,Team_A_TS%_advanced_max,Team_A_TS%_advanced_totals,Team_A_Total,Team_A_USG%_advanced_max,Team_A_USG%_advanced_totals,Team_A_eFG%_advanced_max,Team_A_eFG%_advanced_totals,Team_B_+/-_basic_max,Team_B_3P%_basic_max,Team_B_3P%_basic_totals,Team_B_3PA_basic_max,Team_B_3PA_basic_totals,Team_B_3PAr_advanced_max,Team_B_3PAr_advanced_totals,Team_B_3P_basic_max,Team_B_3P_basic_totals,Team_B_AST%_advanced_max,Team_B_AST%_advanced_totals,Team_B_AST_basic_max,Team_B_AST_basic_totals,Team_B_BLK%_advanced_max,Team_B_BLK%_advanced_totals,Team_B_BLK_basic_max,Team_B_BLK_basic_totals,Team_B_DRB%_advanced_max,Team_B_DRB%_advanced_totals,Team_B_DRB_basic_max,Team_B_DRB_basic_totals,Team_B_DRtg_advanced_max,Team_B_DRtg_advanced_totals,Team_B_FG%_basic_max,Team_B_FG%_basic_totals,Team_B_FGA_basic_max,Team_B_FGA_basic_totals,Team_B_FG_basic_max,Team_B_FG_basic_totals,Team_B_FT%_basic_max,Team_B_FT%_basic_totals,Team_B_FTA_basic_max,Team_B_FTA_basic_totals,Team_B_FT_basic_max,Team_B_FT_basic_totals,Team_B_FTr_advanced_max,Team_B_FTr_advanced_totals,Team_B_GmSc_basic_max,Team_B_MP_advanced_totals,Team_B_ORB%_advanced_max,Team_B_ORB%_advanced_totals,Team_B_ORB_basic_max,Team_B_ORB_basic_totals,Team_B_ORtg_advanced_max,Team_B_ORtg_advanced_totals,Team_B_OT5,Team_B_OT6,Team_B_OT7,Team_B_OT8,Team_B_PF_basic_max,Team_B_PF_basic_totals,Team_B_PTS_basic_max,Team_B_PTS_basic_totals,Team_B_Q1,Team_B_Q2,Team_B_Q3,Team_B_Q4,Team_B_STL%_advanced_max,Team_B_STL%_advanced_totals,Team_B_STL_basic_max,Team_B_STL_basic_totals,Team_B_TOV%_advanced_max,Team_B_TOV%_advanced_totals,Team_B_TOV_basic_max,Team_B_TOV_basic_totals,Team_B_TRB%_advanced_max,Team_B_TRB%_advanced_totals,Team_B_TRB_basic_max,Team_B_TRB_basic_totals,Team_B_TS%_advanced_max,Team_B_TS%_advanced_totals,Team_B_Total,Team_B_USG%_advanced_max,Team_B_USG%_advanced_totals,Team_B_eFG%_advanced_max,Team_B_eFG%_advanced_totals,Team_A_win,Team_A_home
0,BOS,CLE,BOS_vs_CLE,2009-10-27,2010,11.0,1.0,0.474,6.0,19.0,1.0,0.264,3.0,9.0,52.7,62.5,10.0,20.0,8.2,15.1,3.0,8.0,33.3,84.2,10.0,32.0,107.0,99.3,0.6,0.444,16.0,72.0,6.0,32.0,1.000,0.846,10.0,26.0,9.0,22.0,4.0,0.361,18.0,240.0,13.6,18.8,3.0,6.0,125.0,106.0,0.0,0.0,0.0,0.0,5.0,27.0,23.0,95.0,21,30,21,23,4.7,10.0,3.0,9.0,28.7,16.1,3.0,16.0,20.7,54.3,11.0,38.0,0.725,0.569,95,25.2,100.0,0.7,0.507,3.0,0.667,0.353,9.0,17.0,0.409,0.243,4.0,6.0,52.9,58.6,8.0,17.0,8.1,17.0,4.0,9.0,52.1,81.3,10.0,26.0,115.0,106.6,0.545,0.414,22.0,

In [8]:
df = df.sort_values(by=['matchup_id', 'date']).reset_index(drop=True)
df['Team_A_win_next'] = df.groupby('matchup_id')['Team_A_win'].shift(-1)
df.head(6)

,Team_A,Team_B,matchup_id,date,season,Team_A_+/-_basic_max,Team_A_3P%_basic_max,Team_A_3P%_basic_totals,Team_A_3PA_basic_max,Team_A_3PA_basic_totals,Team_A_3PAr_advanced_max,Team_A_3PAr_advanced_totals,Team_A_3P_basic_max,Team_A_3P_basic_totals,Team_A_AST%_advanced_max,Team_A_AST%_advanced_totals,Team_A_AST_basic_max,Team_A_AST_basic_totals,Team_A_BLK%_advanced_max,Team_A_BLK%_advanced_totals,Team_A_BLK_basic_max,Team_A_BLK_basic_totals,Team_A_DRB%_advanced_max,Team_A_DRB%_advanced_totals,Team_A_DRB_basic_max,Team_A_DRB_basic_totals,Team_A_DRtg_advanced_max,Team_A_DRtg_advanced_totals,Team_A_FG%_basic_max,Team_A_FG%_basic_totals,Team_A_FGA_basic_max,Team_A_FGA_basic_totals,Team_A_FG_basic_max,Team_A_FG_basic_totals,Team_A_FT%_basic_max,Team_A_FT%_basic_totals,Team_A_FTA_basic_max,Team_A_FTA_basic_totals,Team_A_FT_basic_max,Team_A_FT_basic_totals,Team_A_FTr_advanced_max,Team_A_FTr_advanced_totals,Team_A_GmSc_basic_max,Team_A_MP_advanced_totals,Team_A_ORB%_advanced_max,Team_A_ORB%_advanced_totals,Team_A_ORB_basic_max,Team_A_ORB_basic_totals,Team_A_ORtg_advanced_max,Team_A_ORtg_advanced_totals,Team_A_OT5,Team_A_OT6,Team_A_OT7,Team_A_OT8,Team_A_PF_basic_max,Team_A_PF_basic_totals,Team_A_PTS_basic_max,Team_A_PTS_basic_totals,Team_A_Q1,Team_A_Q2,Team_A_Q3,Team_A_Q4,Team_A_STL%_advanced_max,Team_A_STL%_advanced_totals,Team_A_STL_basic_max,Team_A_STL_basic_totals,Team_A_TOV%_advanced_max,Team_A_TOV%_advanced_totals,Team_A_TOV_basic_max,Team_A_TOV_basic_totals,Team_A_TRB%_advanced_max,Team_A_TRB%_advanced_totals,Team_A_TRB_basic_max,Team_A_TRB_basic_totals,Team_A_TS%_advanced_max,Team_A_TS%_advanced_totals,Team_A_Total,Team_A_USG%_advanced_max,Team_A_USG%_advanced_totals,Team_A_eFG%_advanced_max,Team_A_eFG%_advanced_totals,Team_B_+/-_basic_max,Team_B_3P%_basic_max,Team_B_3P%_basic_totals,Team_B_3PA_basic_max,Team_B_3PA_basic_totals,Team_B_3PAr_advanced_max,Team_B_3PAr_advanced_totals,Team_B_3P_basic_max,Team_B_3P_basic_totals,Team_B_AST%_advanced_max,Team_B_AST%_advanced_totals,Team_B_AST_basic_max,Team_B_AST_basic_totals,Team_B_BLK%_advanced_max,Team_B_BLK%_advanced_totals,Team_B_BLK_basic_max,Team_B_BLK_basic_totals,Team_B_DRB%_advanced_max,Team_B_DRB%_advanced_totals,Team_B_DRB_basic_max,Team_B_DRB_basic_totals,Team_B_DRtg_advanced_max,Team_B_DRtg_advanced_totals,Team_B_FG%_basic_max,Team_B_FG%_basic_totals,Team_B_FGA_basic_max,Team_B_FGA_basic_totals,Team_B_FG_basic_max,Team_B_FG_basic_totals,Team_B_FT%_basic_max,Team_B_FT%_basic_totals,Team_B_FTA_basic_max,Team_B_FTA_basic_totals,Team_B_FT_basic_max,Team_B_FT_basic_totals,Team_B_FTr_advanced_max,Team_B_FTr_advanced_totals,Team_B_GmSc_basic_max,Team_B_MP_advanced_totals,Team_B_ORB%_advanced_max,Team_B_ORB%_advanced_totals,Team_B_ORB_basic_max,Team_B_ORB_basic_totals,Team_B_ORtg_advanced_max,Team_B_ORtg_advanced_totals,Team_B_OT5,Team_B_OT6,Team_B_OT7,Team_B_OT8,Team_B_PF_basic_max,Team_B_PF_basic_totals,Team_B_PTS_basic_max,Team_B_PTS_basic_totals,Team_B_Q1,Team_B_Q2,Team_B_Q3,Team_B_Q4,Team_B_STL%_advanced_max,Team_B_STL%_advanced_totals,Team_B_STL_basic_max,Team_B_STL_basic_totals,Team_B_TOV%_advanced_max,Team_B_TOV%_advanced_totals,Team_B_TOV_basic_max,Team_B_TOV_basic_totals,Team_B_TRB%_advanced_max,Team_B_TRB%_advanced_totals,Team_B_TRB_basic_max,Team_B_TRB_basic_totals,Team_B_TS%_advanced_max,Team_B_TS%_advanced_totals,Team_B_Total,Team_B_USG%_advanced_max,Team_B_USG%_advanced_totals,Team_B_eFG%_advanced_max,Team_B_eFG%_advanced_totals,Team_A_win,Team_A_home,Team_A_win_next
0,ATL,BOS,ATL_vs_BOS,2009-11-13,2010,23.0,0.667,0.267,6.0,15.0,0.500,0.179,2.0,4.0,24.0,50.0,6.0,19.0,12.3,10.3,3.0,6.0,57.2,83.8,9.0,31.0,105.0,98.2,0.667,0.452,21.0,84.0,9.0,38.0,1.000,0.739,7.0,23.0,5.0,17.0,0.500,0.274,14.8,240.0,27.5,41.0,4.0,16.0,155.0,110.7,0.0,0.0,0.0,0.0,4.0,20.0,24.0,97.0,24,17,31,25,5.2,8.0,3.0,7.0,66.7,12.1,3.0,13.0,28.2,61.8,13.0,47.0,0.667,0.515,97,33.9,100.0,0.667,0.476,3.0,0.2,0.067,5.0,15.0,1.000,0.205,1.0,1.0,37.6,55.6,9.0,20.0,9.7,5.8,3.0,4.0,29.4,59.0,7.0,23.0,121.0,110

In [9]:
playoff_start_dates = {
    2010: '2010-04-17',
    2011: '2011-04-16',
    2012: '2012-04-28',
    2013: '2013-04-20',
    2014: '2014-04-19',
    2015: '2015-04-18',
    2016: '2016-04-16',
    2017: '2017-04-15',
    2018: '2018-04-14',
    2019: '2019-04-13',
    2020: '2020-08-17',
    2021: '2021-05-22',
    2022: '2022-04-16',
    2023: '2023-04-15',
    2024: '2024-04-20'
}
df['playoff_game'] = df.apply(lambda row: 1 if row['date'] >= pd.to_datetime(playoff_start_dates.get(row['season'])) else 0, axis=1)
df = df.drop(columns=['season'])
df['playoff_game'].value_counts()

playoff_game
0    17914
1     1247
Name: count, dtype: int64

In [10]:
#df.head(10)
print(f'Total missing values: {df.isnull().sum().sum()}')
print(f"Total missing values in Team_win_next label: {df['Team_A_win_next'].isnull().sum()}")
df = df.dropna(axis=0, subset=['Team_A_win_next'])

Total missing values: 522
Total missing values in Team_win_next label: 522


In [11]:
df.columns.tolist()

['Team_A',
 'Team_B',
 'matchup_id',
 'date',
 'Team_A_+/-_basic_max',
 'Team_A_3P%_basic_max',
 'Team_A_3P%_basic_totals',
 'Team_A_3PA_basic_max',
 'Team_A_3PA_basic_totals',
 'Team_A_3PAr_advanced_max',
 'Team_A_3PAr_advanced_totals',
 'Team_A_3P_basic_max',
 'Team_A_3P_basic_totals',
 'Team_A_AST%_advanced_max',
 'Team_A_AST%_advanced_totals',
 'Team_A_AST_basic_max',
 'Team_A_AST_basic_totals',
 'Team_A_BLK%_advanced_max',
 'Team_A_BLK%_advanced_totals',
 'Team_A_BLK_basic_max',
 'Team_A_BLK_basic_totals',
 'Team_A_DRB%_advanced_max',
 'Team_A_DRB%_advanced_totals',
 'Team_A_DRB_basic_max',
 'Team_A_DRB_basic_totals',
 'Team_A_DRtg_advanced_max',
 'Team_A_DRtg_advanced_totals',
 'Team_A_FG%_basic_max',
 'Team_A_FG%_basic_totals',
 'Team_A_FGA_basic_max',
 'Team_A_FGA_basic_totals',
 'Team_A_FG_basic_max',
 'Team_A_FG_basic_totals',
 'Team_A_FT%_basic_max',
 'Team_A_FT%_basic_totals',
 'Team_A_FTA_basic_max',
 'Team_A_FTA_basic_totals',
 'Team_A_FT_basic_max',
 'Team_A_FT_basic_tot

In [12]:
num_games = df.shape[0]
print(f'Total games from the 2009-2010 season to the 2023-2024 season: {num_games}')
num_features = df.shape[1] - 1
print(f'Number of features: {num_features}')
categorical_variables = df.select_dtypes(include=['object', 'category']).columns.tolist()
print(f'Categorical variables: {categorical_variables}')

Total games from the 2009-2010 season to the 2023-2024 season: 18639
Number of features: 159
Categorical variables: ['Team_A', 'Team_B', 'matchup_id']


In [13]:
numerical_features  = ['Team_A_+/-_basic_max',
 'Team_A_3P%_basic_max',
 'Team_A_3P%_basic_totals',
 'Team_A_3PA_basic_max',
 'Team_A_3PA_basic_totals',
 'Team_A_3PAr_advanced_max',
 'Team_A_3PAr_advanced_totals',
 'Team_A_3P_basic_max',
 'Team_A_3P_basic_totals',
 'Team_A_AST%_advanced_max',
 'Team_A_AST%_advanced_totals',
 'Team_A_AST_basic_max',
 'Team_A_AST_basic_totals',
 'Team_A_BLK%_advanced_max',
 'Team_A_BLK%_advanced_totals',
 'Team_A_BLK_basic_max',
 'Team_A_BLK_basic_totals',
 'Team_A_DRB%_advanced_max',
 'Team_A_DRB%_advanced_totals',
 'Team_A_DRB_basic_max',
 'Team_A_DRB_basic_totals',
 'Team_A_DRtg_advanced_max',
 'Team_A_DRtg_advanced_totals',
 'Team_A_FG%_basic_max',
 'Team_A_FG%_basic_totals',
 'Team_A_FGA_basic_max',
 'Team_A_FGA_basic_totals',
 'Team_A_FG_basic_max',
 'Team_A_FG_basic_totals',
 'Team_A_FT%_basic_max',
 'Team_A_FT%_basic_totals',
 'Team_A_FTA_basic_max',
 'Team_A_FTA_basic_totals',
 'Team_A_FT_basic_max',
 'Team_A_FT_basic_totals',
 'Team_A_FTr_advanced_max',
 'Team_A_FTr_advanced_totals',
 'Team_A_GmSc_basic_max',
 'Team_A_MP_advanced_totals',
 'Team_A_ORB%_advanced_max',
 'Team_A_ORB%_advanced_totals',
 'Team_A_ORB_basic_max',
 'Team_A_ORB_basic_totals',
 'Team_A_ORtg_advanced_max',
 'Team_A_ORtg_advanced_totals',
 'Team_A_OT5',
 'Team_A_OT6',
 'Team_A_OT7',
 'Team_A_OT8',
 'Team_A_PF_basic_max',
 'Team_A_PF_basic_totals',
 'Team_A_PTS_basic_max',
 'Team_A_PTS_basic_totals',
 'Team_A_Q1',
 'Team_A_Q2',
 'Team_A_Q3',
 'Team_A_Q4',
 'Team_A_STL%_advanced_max',
 'Team_A_STL%_advanced_totals',
 'Team_A_STL_basic_max',
 'Team_A_STL_basic_totals',
 'Team_A_TOV%_advanced_max',
 'Team_A_TOV%_advanced_totals',
 'Team_A_TOV_basic_max',
 'Team_A_TOV_basic_totals',
 'Team_A_TRB%_advanced_max',
 'Team_A_TRB%_advanced_totals',
 'Team_A_TRB_basic_max',
 'Team_A_TRB_basic_totals',
 'Team_A_TS%_advanced_max',
 'Team_A_TS%_advanced_totals',
 'Team_A_Total',
 'Team_A_USG%_advanced_max',
 'Team_A_USG%_advanced_totals',
 'Team_A_eFG%_advanced_max',
 'Team_A_eFG%_advanced_totals',
 'Team_B_+/-_basic_max',
 'Team_B_3P%_basic_max',
 'Team_B_3P%_basic_totals',
 'Team_B_3PA_basic_max',
 'Team_B_3PA_basic_totals',
 'Team_B_3PAr_advanced_max',
 'Team_B_3PAr_advanced_totals',
 'Team_B_3P_basic_max',
 'Team_B_3P_basic_totals',
 'Team_B_AST%_advanced_max',
 'Team_B_AST%_advanced_totals',
 'Team_B_AST_basic_max',
 'Team_B_AST_basic_totals',
 'Team_B_BLK%_advanced_max',
 'Team_B_BLK%_advanced_totals',
 'Team_B_BLK_basic_max',
 'Team_B_BLK_basic_totals',
 'Team_B_DRB%_advanced_max',
 'Team_B_DRB%_advanced_totals',
 'Team_B_DRB_basic_max',
 'Team_B_DRB_basic_totals',
 'Team_B_DRtg_advanced_max',
 'Team_B_DRtg_advanced_totals',
 'Team_B_FG%_basic_max',
 'Team_B_FG%_basic_totals',
 'Team_B_FGA_basic_max',
 'Team_B_FGA_basic_totals',
 'Team_B_FG_basic_max',
 'Team_B_FG_basic_totals',
 'Team_B_FT%_basic_max',
 'Team_B_FT%_basic_totals',
 'Team_B_FTA_basic_max',
 'Team_B_FTA_basic_totals',
 'Team_B_FT_basic_max',
 'Team_B_FT_basic_totals',
 'Team_B_FTr_advanced_max',
 'Team_B_FTr_advanced_totals',
 'Team_B_GmSc_basic_max',
 'Team_B_MP_advanced_totals',
 'Team_B_ORB%_advanced_max',
 'Team_B_ORB%_advanced_totals',
 'Team_B_ORB_basic_max',
 'Team_B_ORB_basic_totals',
 'Team_B_ORtg_advanced_max',
 'Team_B_ORtg_advanced_totals',
 'Team_B_OT5',
 'Team_B_OT6',
 'Team_B_OT7',
 'Team_B_OT8',
 'Team_B_PF_basic_max',
 'Team_B_PF_basic_totals',
 'Team_B_PTS_basic_max',
 'Team_B_PTS_basic_totals',
 'Team_B_Q1',
 'Team_B_Q2',
 'Team_B_Q3',
 'Team_B_Q4',
 'Team_B_STL%_advanced_max',
 'Team_B_STL%_advanced_totals',
 'Team_B_STL_basic_max',
 'Team_B_STL_basic_totals',
 'Team_B_TOV%_advanced_max',
 'Team_B_TOV%_advanced_totals',
 'Team_B_TOV_basic_max',
 'Team_B_TOV_basic_totals',
 'Team_B_TRB%_advanced_max',
 'Team_B_TRB%_advanced_totals',
 'Team_B_TRB_basic_max',
 'Team_B_TRB_basic_totals',
 'Team_B_TS%_advanced_max',
 'Team_B_TS%_advanced_totals',
 'Team_B_Total',
 'Team_B_USG%_advanced_max',
 'Team_B_USG%_advanced_totals',
 'Team_B_eFG%_advanced_max',
 'Team_B_eFG%_advanced_totals']



In [14]:
for col in numerical_features:
    correlation, p_value = sps.pointbiserialr(df[col], df['Team_A_win_next'])
    print(f'Correlation between {col} and Team_A_win_next: {correlation}, p: {p_value}')

Correlation between Team_A_+/-_basic_max and Team_A_win_next: 0.11473666025894581, p: 1.1888993224873088e-55
Correlation between Team_A_3P%_basic_max and Team_A_win_next: 0.006338809084527497, p: 0.3868440769710476
Correlation between Team_A_3P%_basic_totals and Team_A_win_next: 0.02958024500686766, p: 5.366865624266705e-05
Correlation between Team_A_3PA_basic_max and Team_A_win_next: 0.041944599713186106, p: 1.012558279201593e-08
Correlation between Team_A_3PA_basic_totals and Team_A_win_next: 0.03227567479736122, p: 1.046767451047507e-05
Correlation between Team_A_3PAr_advanced_max and Team_A_win_next: 0.0313709223770806, p: 1.838326324559446e-05
Correlation between Team_A_3PAr_advanced_totals and Team_A_win_next: 0.03585047941139106, p: 9.794535791031125e-07
Correlation between Team_A_3P_basic_max and Team_A_win_next: 0.04559323976788893, p: 4.741628177277579e-10
Correlation between Team_A_3P_basic_totals and Team_A_win_next: 0.04554370083369676, p: 4.950935802064832e-10
Correlation

/Users/sarmadahmed/anaconda3/lib/python3.11/site-packages/scipy/stats/_stats_py.py:4781: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))


The Point biserial correlation is calculated between each numerical features and the target label Team_A_win_next. Point biserial correlation measures the 

In [15]:
df = df.drop(columns=['Team_A_USG%_advanced_totals', 'Team_B_USG%_advanced_totals'])
#Correlation and p values both nan, values of USG%_advanced_totals constant 

In [16]:
for col in numerical_features:
    distance_correlation = dcor.distance_correlation(df[col], df['Team_A_win_next'])
    print(f'Correlation between {col} and Team_A_win_next: {dcor_val:.4f}')

/Users/sarmadahmed/anaconda3/bin/python
